In [ ]:
!pip install requests beautifulsoup4

In [ ]:
import requests
from bs4 import BeautifulSoup
import uuid

In [ ]:
subscription_key = ""
endpoint = "https://api.cognitive.microsofttranslator.com/"
location = ""

In [ ]:
# Check if subscription_key is already set and has a value, if not, prompt for it
if 'subscription_key' not in globals() or not globals().get('subscription_key'):
    subscription_key = input("Por favor, entre com o Azure Translator subscription key: ")
    print("Subscription key configurado.")
else:
    print("Subscription key já configurado.")

# Check if location is already set and has a value, if not, prompt for it
if 'location' not in globals() or not globals().get('location'):
    location = input("Por favor, entre com o Azure Translator service region (e.g., 'eastus2'): ")
    print("Location configurado.")
else:
    print("Location já configurado")

In [ ]:
def extrair_texto_da_url(url):
  response = requests.get(url)

  if response.status_code != 200:
    print(f"Erro ao acessar a URL. Código de status: {response.status_code}")
    return None

  soup = BeautifulSoup(response.text, 'html.parser')
  titulo = soup.find(class_='crayons-article__header__meta').h1.text.strip()
  conteudo = soup.find(class_='crayons-article__main').text.strip()

  return titulo, conteudo

In [ ]:
def traduzir(texto, linguagem):
  path = 'translate'
  constructed_url = endpoint + path

  headers = {
      'Ocp-Apim-Subscription-Key': subscription_key,
      'Ocp-Apim-Subscription-Region': location,
      'Content-type': 'application/json',
      'X-ClientTraceId': str(uuid.uuid4())
  }
  body = [{
      'text': texto
  }]
  params = {
      'api-version': '3.0',
      'to': linguagem,
      'from': 'en'
  }
  request = requests.post(constructed_url, params=params, headers=headers, json=body)
  response = request.json()

  return response[0]['translations'][0]['text']


In [ ]:
url_input = input("Por favor, insira a URL do site (padrão: https://dev.to): ")
url = url_input.strip() if url_input.strip() else 'https://dev.to/aaron_rose_0787cc8b4775a0/the-secret-life-of-python-the-silent-type-type-casting-5c57'

# Validate if the URL contains 'https://dev.to/'
if 'https://dev.to/' not in url:
    print("Erro: A URL deve conter 'https://dev.to/'. Por favor, insira uma URL válida.")
else:
    linguagem_destino_input = input("Por favor, insira a linguagem destino (padrão: 'pt-br'): ")
    linguagem_destino = linguagem_destino_input.strip() if linguagem_destino_input.strip() else 'pt-br'

    print(f"Extraindo conteúdo de: {url}")
    titulo, conteudo = extrair_texto_da_url(url)

    if conteudo:
        print(f"Traduzindo conteúdo para: {linguagem_destino}")
        conteudo_traduzido = traduzir(conteudo, linguagem_destino)
        print("Conteúdo Traduzido:")
        print(conteudo_traduzido)
    else:
        print("Não foi possível extrair o conteúdo para tradução.")